First, some philosophy:

The CatSim stack principally exists to create catalogs from simulated universes.  The default simulated universe that CatSim accesses lives on a machine at the University of Washington called "fatboy".  This simulated universe is really a <b>database</b> that consists of a distribution of galaxies drawn from the Millennium N-body simulation, and Milky Way stars generated with the GalFast software.  The most up-to-date documentation of what fatboy provides is here

https://confluence.lsstcorp.org/display/SIM/Database+Contents+--+Catalog+Simulations

https://confluence.lsstcorp.org/display/SIM/Database+Schema

Whenever this notebook refers to "querying the database," it is referring to this database on fatboy (it is possible, in principle, to connect CatSim to your own database containing your own simulated universe; that is beyond the scope of this demo).

<b>See the link below for instructions on how to access the fatboy database</b>

https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database

Even though catalogs are generated by querying fatboy's database, CatSim is designed so that the user should never have to write any raw SQL queries.  This is due to the way the catalog-generating classes in CatSim have been written.  In broad strokes:

* The user instantiates a daughter of the <b>`InstanceCatalog`</b> class which is in charge of actually writing the catalog.  This is the class that contains information regarding what astronomical objects and what data regarding those objects should be written to the catalog.


* The user passes the InstanceCatalog an instantiation of a daughter of the <b>`CatalogDBObject`</b> class.  The `CatalogDBObject` class is the class which actually manipulates the connection to fatboy.  Specific `CatalogDBobject` classes are written to connect to specific tables in the fatboy database.  Thus, there is one `CatalogDBObject` class for galaxies, a different `CatalogDBObject` class for Solar System objects, a different `CatalogDBObject` for main sequence stars, a different `CatalogDBObjec`t for white dwarfs, etc.  The full list of available `CatalogDBObject` daughter classes is here  https://confluence.lsstcorp.org/display/SIM/Database+Schema.  `CatalogDBObject` daughter classes provide one other purpose: naming conventions for data columns vary between fatboy and the `InstanceCatalog` classes (and, indeed, between tables in fatboy).  Declination is called `dec` in the galaxy table but `decl` in the star tables.  `CatalogDBObject` classes provide simple mappings that smooth over these differences and put all of the database columns into a uniform syntax.


* The user also passes the `InstanceCatalog` an instantiation of the <b>`ObservationMetaData`</b> class.  This is the class which contains data about the state of the simulated telescope.  For the purposes of generating a catalog, the `ObservationMetaData` provides the RA and Dec at which the telescope is pointed as well as the size and shape of its field of view (i.e. it controls the question "which objects in fatboy's database are actually seen by my telescope and thus should be written to my catalog?").

When you ask your `InstanceCatalog` to write itself out, what actually happens is that the `CatalogDBObject` performs a query on fatboy using information from the `ObservationMetaData` to tell it which objects to return and information from the `InstanceCatalog` to tell it what data columns associated with those objects to return.

Let's do that now.

PS: make sure you have setup all of the sims packages before attempting to run this notebook.  If you haven't, exit the notebook and run

    source loadLSST.bash
    setup sims_catUtils -t sims
    setup sims_GalSimInterface -t sims
    
in the shell from which you want to run this notebook

PPS: if this notebook does not answer your questions, you can find more tutorials in 

    ~lsst_home/yourOperatingSystem/sims_catUtils/yourVersion/examples/tutorials/

where `yourOperatingSyste`m will be 'DarwinX86' for Mac users and 'Linux64' for Linux users.  `yourVersion` will be a Git SHA-1 indicating the state of the `sims_catUtils` code when you installed it.

In [ ]:
import numpy
from lsst.sims.catalogs.definitions import InstanceCatalog

class simpleStarCatalog(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'sedFilename']
    
    transformations = {'raJ2000':numpy.degrees, 'decJ2000':numpy.degrees}

from lsst.sims.utils import ObservationMetaData

myObsMetadata = ObservationMetaData(pointingRA=45.0, pointingDec=-10.0,
                                    boundType='circle', boundLength=0.02)

from lsst.sims.catUtils.baseCatalogModels import StarObj

starTableConnection = StarObj()

myCatalog = simpleStarCatalog(starTableConnection,
                              obs_metadata = myObsMetadata)

myCatalog.write_catalog('test_catalog.txt')

readCatalog = open('test_catalog.txt', 'r').readlines()

!cat test_catalog.txt

So what just happened?

    import numpy
    from lsst.sims.catalogs.defintions import InstanceCatalog
    class simpleStarCatalog(InstanceCatalog):

        column_outputs = ['raJ2000', 'decJ2000', 'sedFilename']
    
        transformations = {'raJ2000':numpy.degrees,
                           'decJ2000':numpy.degrees}

This here we defined a daughter class of the class `InstanceCatalog`.  We told it what columns we wanted the catalog to contain by defining the `column_outputs` member variable, and we told it that we wanted RA and Dec to be written in degrees using the `transformations` member variable.  <b>Note: CatSim philosophy is that angles are all handled internally in radians but externally in degrees.</b>  Thus, any time you have to provide an angle, it should be in degrees.  However, the angles are immediately translated into radians inside the code.  Because our `InstanceCatalog` daughter class takes those internally-handled angles and writes them to disk, we have to explicitly convert them to degrees.

    from lsst.sims.utils import ObservationMetaData
    myObsMetadata = ObservationMetaData(unrefractedRA=45.0,
                                unrefractedDec=-10.0,
                                boundType='circle', boundLength=0.02)
                                
Here, we constructed an `ObservationMetaData` class centered on (RA, Dec) = (45, -10) in degrees (because this is an angle exposed to the user).  We also told it that we were going to want a circular field of view with a radius of 0.02 degrees.  Currently, `ObservationMetaData` also supports `boundType='box'` in which case `boundLength` is either the half-side length of a square or a tuple `(halfSideLengthRA, halfSideLengthDec)`.

    from lsst.sims.catUtils.baseCatalogModels import StarObj
    starTableConnection = StarObj()

This is where we instantiated our `CatalogDBObject` daughter class.  In this case, we instantiated the class `StarObj` which, we see from

https://confluence.lsstcorp.org/display/SIM/Database+Schema

provides a union of main sequence, red giant branch, white dwarf, RR Lyrae, and blue horizontal branch stars.

We combined the `InstanceCatalog`, `CatalogDBObject`, and `ObservationMetaData` by instantiating our catalog class, passing the `CatalogDBObject` as an argument and the `ObservationMetaData` as a `kwarg`.

    myCatalog = simpleStarCatalog(starTableConnection,
                                  obs_metadata = myObsMetadata)

This ultimately allowed us to call `myCatalog.write_catalog('test_catalog.txt')` and write the catalog to a file `test_catalog.txt'`.  This is all one absolutely has to do to get a simulated catalog out of CatSim.

<b>Note:</b> `InstanceCatalog` is associated with a meta-class that keeps a registry of all defined `InstanceCatalog` daughter classes.  This meta-class will throw an exception if you try to run the cell above (or any cell that defines an `InstanceCatalog` daughter class) more than once.  It will warn you that you are trying to define a class that has already been defined.  If you need to re-run a catalog-class-defining cell again, you will have to restart the kernel of this iPython notebook.  The same warning applies for cells that define `CatalogDBObject` daugther classes.

# Where do InstanceCatalogs get their data?

In the simple example above, all of the data from our catalog was provided by the `CatalogDBObject` class.  The database on fatboy contains very basic information about all of the objects in our simulated universe.  The class `StarObj` contains simple functions that map that data into the syntax of our `InstanceCatalog`.  Calling `write_catalog()` in this case literally just peformed an SQL query and returned the results.  Indeed, we can see this by doing

In [ ]:
starTableConnection.show_mapped_columns()

This command prints to screen all of the database columns that `starTableConnection` natively provides and, indeed, we see that `raJ2000`, `dec2000`, and `sedFilename` all exist in the (mapped) database table provided by `starTableConnection`.  Suppose, however, that we wanted to include data not provided by the `CatalogDBObject` class.  For example, the star database tables contain mean RA and Dec and proper motions for all of the stars in fatboy.  What if we want to write a catalog that writes out stars to their proper motion-corrected positions on the sky?  In that case, we add the new data columns by adding getter methods to our `InstanceCatalog` daughter class.

In [ ]:
class demoProperMotionCatalog(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'correctedRA', 'correctedDec']

    transformations = {'raJ2000':numpy.degrees,
                       'decJ2000':numpy.degrees,
                       'correctedRA':numpy.degrees,
                       'correctedDec':numpy.degrees}
    
    def get_correctedRA(self):
        dt = self.obs_metadata.mjd.TAI - 51544.0
        ra = self.column_by_name('raJ2000')
        speed = self.column_by_name('properMotionRa')
        return ra + speed*dt
    
    def get_correctedDec(self):
        dt = self.obs_metadata.mjd.TAI - 51544.0
        dec = self.column_by_name('decJ2000')
        speed = self.column_by_name('properMotionDec')
        return dec + speed*dt

In [ ]:
myObsMetadata = ObservationMetaData(pointingRA=45.0, pointingDec=-10.0,
                                    boundType='circle', boundLength=0.02,
                                    mjd=57098.0)

myProperMotionCat = demoProperMotionCatalog(starTableConnection,
                                            obs_metadata=myObsMetadata)

myProperMotionCat.write_catalog('proper_motion_example.txt')
!cat proper_motion_example.txt

How did we do that?

In `column_outputs` we asked the `InstanceCatalog` to write the columns `correctedRA` and `correctedDec` which do not exist in the database table provided by `starTableConnection`.  In order for the `InstanceCatalog` to calculate these columns on the fly, we had to define getter methods in our `InstanceCatalog` daughter class.  <b>Literally, the InstanceCatalog inspects itself, looking for methods named `get_columnName()`.</b>  These methods need to return numpy arrays whose elements are in the same order as the rows in the database table.  These getter methods are called whenever the `InstanceCatalog` writes itself to disk.  They are also called whenever another getter method calls `self.column_by_name('columnName')` (which is how getter method columns can be calculated using columns defined in the database).

Obviously, there are a few problems with what we did above.  The first is that it was inefficient.  We used two getters when we could have used one.  The `@compound` decorator allows a single getter method to return two columns as follows.

In [ ]:
from lsst.sims.catalogs.decorators import compound

class demoProperMotionCatalog2(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'correctedRA', 'correctedDec']

    transformations = {'raJ2000':numpy.degrees,
                       'decJ2000':numpy.degrees,
                       'correctedRA':numpy.degrees,
                       'correctedDec':numpy.degrees}
    
    @compound('correctedRA', 'correctedDec')
    def get_correctedCoords(self):
        dt = self.obs_metadata.mjd.TAI - 51544.0
        ra = self.column_by_name('raJ2000')
        speedRa = self.column_by_name('properMotionRa')
        dec = self.column_by_name('decJ2000')
        speedDec = self.column_by_name('properMotionDec')
        
        #The new columns must be returned as rows of a numpy array
        #in the order that they were specified to the @compound getter
        return numpy.array([ra + speedRa*dt, dec + speedDec*dt])

In [ ]:
myObsMetadata = ObservationMetaData(pointingRA=45.0, pointingDec=-10.0,
                                    boundType='circle', boundLength=0.02,
                                    mjd=57098.0)

myProperMotionCat = demoProperMotionCatalog2(starTableConnection,
                                            obs_metadata=myObsMetadata)

myProperMotionCat.write_catalog('proper_motion_example.txt')
!cat proper_motion_example.txt

The other thing that is wrong with this catalog is that we have not applied proper motion correctly.  This is just a cartoon which I simplified for the purposes of this demo.  Forgive me.  The correct calculation is much more complicated.  Fortunately, we have already implemented that calculation in the CatSim stack in the form of the Astrometry <b>mixin</b>.

Mixins are Python classes that exist solely to define methods to be inherited by other classes.  In the case of CatSim, we use mixins to provide getter methods to be inherited by `InstanceCatalog` daughter classes.  Many mixins exist in CatSim to provide calculated columns to `InstanceCatalog`s.  They can be listed by with

```
from lsst.sims.catUtils import mixins

dir(mixins)
```

One of the mixins -- `AstrometryStars` -- provides getters for the columns `raObserved` and `decObserved` which apply proper motion, parallax, precession, nutation, aberration, and refraction by the atmosphere to RA and Dec to get the observed geocentric RA, Dec (as opposed to the usual coordinates in the International Celestial Reference System).  We can include these columns in our InstanceCatalog like this:

In [ ]:
from lsst.sims.catUtils.mixins import AstrometryStars

class astrometricCatalog(InstanceCatalog, AstrometryStars):
    column_outputs = ['raJ2000', 'decJ2000', 'raObserved', 'decObserved']
    transformations = {'raJ2000':numpy.degrees,
                       'decJ2000':numpy.degrees,
                       'raObserved':numpy.degrees,
                       'decObserved':numpy.degrees}

In [ ]:
myObsMetadata = ObservationMetaData(pointingRA=45.0, pointingDec=-10.0,
                                    boundType='circle', boundLength=0.02,
                                    mjd=57098.0)

myAstrometricCat = astrometricCatalog(starTableConnection,
                                            obs_metadata=myObsMetadata)

myAstrometricCat.write_catalog('astrometry_example.txt')
!cat astrometry_example.txt

Be defining `astrometricCatalog` to inherit both from `InstanceCatalog` and `AstrometryStars`, we made sure that the `astrometricCatalog` combined the basic functionality of the `InstanceCatalog` with the getter methods provided by `AstrometryStars`, giving us access to the columns `raObserved` and `decObserved`.

So far, we have seen how an `InstanceCatalog` can get columns either directly from the database (as translated through the `CatalogDBObject`) or from getter methods.  There is one other way to provide columns to the `InstanceCatalog`: default values.

In [ ]:
class defaultColumnExampleCatalog(InstanceCatalog):
    column_outputs = ['raJ2000', 'decJ2000', 'fudgeFactor1', 'fudgeFactor2']
    
    transformations = {'raJ2000':numpy.degrees, 'decJ2000':numpy.degrees}
    
    default_columns = [('fudgeFactor1', 1.1, float),
                       ('fudgeFactor2', 'hello', (str,5))]

In [ ]:
fudgeCat = defaultColumnExampleCatalog(starTableConnection,
                                       obs_metadata=myObsMetadata)

fudgeCat.write_catalog('default_example.txt')
!cat default_example.txt

The member variable `default_columns` allows you to assign default values to columns that do not exist either in the `CatalogDBObject` or as getter methods.  `default_columns` is a list of tuples.  Each tuple corresponds to a different column being defaulted.  The first entry in the tuple is the name of the column.  The second entry is its value.  The third entry is its data type.

# Getting realistic ObservationMetaData

In all of the examples so far, we have been creating our `ObservationMetaData` objects by hand.  It is, however, possible to create `ObservationMetaData` objects that are based on actual actual pointings generated with the LSST Operations Simulator (or OpSim).  We do this using the `ObservationMetaDataGenerator` class defined in the `sims_catUtils` package.  We will use a cartoon OpSim output included with the simulations software stack.  More realistic OpSim outputs can be downloaded from

https://www.lsst.org/scientists/simulations/opsim/opsim-v335-benchmark-surveys

In [ ]:
import os
import eups
from lsst.sims.catUtils.utils import ObservationMetaDataGenerator

opsimPath = os.path.join(eups.productDir('sims_data'),'OpSimData')
opsimDB = os.path.join(opsimPath,'opsimblitz1_1133_sqlite.db')

#you need to provide ObservationMetaDataGenerator with the connection
#string to an OpSim output database.  This is the connection string
#to a test database that comes when you install CatSim.
obs_generator = ObservationMetaDataGenerator(database=opsimDB, driver='sqlite')

The `ObservationMetaDataGenerator` allows you to query OpSim pointings according to physical criteria.  It then returns ObservationMetaData based on the pointings that meet your criteria.  For example, let's say we wanted 10 pointings with 5 < RA < 8 degrees

In [ ]:
obsMetaDataResults = obs_generator.getObservationMetaData(limit=10, fieldRA=(5.0, 8.0))

for obs_metadata in obsMetaDataResults:    
    print obs_metadata.pointingRA

To see the full list of parameters you can query OpSimPointings on using the ObservationMetaDataGenerator

In [ ]:
help(ObservationMetaDataGenerator.getObservationMetaData)

# Generating Images with PhoSim

PhoSim operates as a piece of software totally independent from CatSim.  The documentation for PhoSim can be found here:

https://confluence.lsstcorp.org/pages/viewpage.action?pageId=4129126

To convert CatSim catalogs into PhoSim images, you must use CatSim to write your catalog in a format that PhoSim expects and then (by hand) feed that catalog into PhoSim. Fortunately, we have written scripts that know how to format CatSim catalogs for just this purpose.  The script

    sims_catUtils/examples/generatePhosimInput.py

will write out a catalog `phoSim_example.txt` that one can run through phoSim using the command

    phosim phosim_example.txt

to get out FITS images.  Below, we will step through `generatePhosimInput.py`, explaining how the script operates.

In [ ]:
from __future__ import with_statement
from lsst.sims.utils import ObservationMetaData
from lsst.sims.catalogs.definitions import InstanceCatalog
from lsst.sims.catalogs.db import CatalogDBObject
from lsst.sims.catUtils.exampleCatalogDefinitions import \
        (PhoSimCatalogPoint, PhoSimCatalogSersic2D, PhoSimCatalogZPoint,
         DefaultPhoSimHeaderMap)

from lsst.sims.catUtils.baseCatalogModels import *

Most of what is above ought to be familiar.  `PhoSimCatalogPoint`, `PhoSimCatalogSersic2D` and `PhoSimCatalogZPoint` are `InstanceCatalog` daughter classes written to format catalogs of point sources, sersic profiles, and extra-galactic point sources (i.e. AGN) respectively for input into PhoSim

In [ ]:
obs_metadata_list = obs_generator.getObservationMetaData(obsHistID=10)
obs_metadata = obs_metadata_list[0]

Now we are in a difficult position.  We would like to generate PhoSim images with all kinds of astronomical objects included.  However, the way the `CatalogDBObject` classes are written, each class only interfaces with one database table, i.e. only one kind of astronomical object.  Fortunately, `InstanceCatalog` provides functionality that allows you to write multiple catalogs to a single file.  The code below loops over all of the different varieties of stellar object and writes them to the same PhoSim-ready catalog file.

<b>Note:</b> The cells below will run for a long time as they try to find all of the galaxies in one particular LSST field of view (there are a lot of them).  You can adjust this by reducing the size of your field of view with the `ObservationMetaData.boundLength` attribute.  `boundLength` is the radius of the field of view in degrees (1.75 is the nominal LSST field of view radius).

In [ ]:
starObjNames = ['msstars', 'bhbstars', 'wdstars', 'rrlystars', 'cepheidstars']

doHeader= True
for starName in starObjNames:
    stars = CatalogDBObject.from_objid(starName)
    star_phoSim=PhoSimCatalogPoint(stars,obs_metadata=obs_metadata) #the class for phoSim input files
                                                                #containing point sources
    if (doHeader):
        star_phoSim.phoSimHeaderMap = DefaultPhoSimHeaderMap
        with open("phoSim_example.txt","w") as fh:
            star_phoSim.write_header(fh)
        doHeader = False

    #below, write_header=False prevents the code from overwriting the header just written
    #write_mode = 'a' allows the code to append the new objects to the output file, rather
    #than overwriting the file for each different class of object.
    star_phoSim.write_catalog("phoSim_example.txt",write_mode='a',write_header=False,chunk_size=20000)


Now, we will do the same thing for galaxy components, using the `PhoSimCatalogSersic2D` `InstanceCatalog` class for galaxy bulges and disks and using `PhoSimCatalogZPoint` for AGNs. 

In [ ]:
gals = CatalogDBObject.from_objid('galaxyBulge')

#now append a bunch of objects with 2D sersic profiles to our output file
galaxy_phoSim = PhoSimCatalogSersic2D(gals, obs_metadata=obs_metadata)
galaxy_phoSim.write_catalog("phoSim_example.txt",write_mode='a',write_header=False,chunk_size=20000)

gals = CatalogDBObject.from_objid('galaxyDisk')
galaxy_phoSim = PhoSimCatalogSersic2D(gals, obs_metadata=obs_metadata)
galaxy_phoSim.write_catalog("phoSim_example.txt",write_mode='a',write_header=False,chunk_size=20000)

gals = CatalogDBObject.from_objid('galaxyAgn')

#PhoSimCatalogZPoint is the phoSim input class for extragalactic point sources (there will be no parallax
#or proper motion)
galaxy_phoSim = PhoSimCatalogZPoint(gals, obs_metadata=obs_metadata)
galaxy_phoSim.write_catalog("phoSim_example.txt",write_mode='a',write_header=False,chunk_size=20000)


We now have a catalog file phoSim_example.txt that contains all the information PhoSim needs to generate a simulated LSST focal plane (albeit with a very small populated field of view; <b> note that our `ObservationMetaData` only drew a circle with a radius of 0.05 degrees.</b>

The output generated by PhoSim is described here:

https://confluence.lsstcorp.org/display/PHOSIM/Output

# Generating Images with GalSim

Unlike PhoSim, GalSim can be run seamlessly with the CatSim stack.  The code governing the CatSim-GalSim interface resides in the package

    sims_GalSimInterface

The basic philosophy of the CatSim-GalSim interface is as follows:


* The user instantiates a GalSim catalog class as defined in `sims_GalSimInterface/python/lsst/sims/GalSimInterface/galSimCatalogs.py`.  These are `InstanceCatalog` daughter classes designed to amass the information that GalSim needs to generate images of a field of view.  Like any `InstanceCatalog` daughter class, these classes require a `CatalogDBobject` and an `ObservationMetaData.`


* Each GalSim catalog class contains in it an instantiation of the class `GalSimInterpreter` defined in `sims_GalSimInterface/python/lsst/sims/GalSimInterface/galSimInterpreter.py`.  As the user iterates over the GalSim catalog (for instance, by calling `write_catalog`), astronomical objects are passed to the `GalSimInterpreter`.  The `GalSimInterpreter` determines which chips in the camera the each astronomical object falls on and adds that astronomical object to the corresponding chips' FITS files.


* Once the user has iterated over the catalog, the method `write_images` (which belongs to the GalSim Catalog class) will write out FITS images for all of the chips in the camera, based on the objects that have been passed through the `GalSimInterpreter`.

Examples of how to use this system can be found in

    sims_GalSimInterface/examples/

In [ ]:
from lsst.sims.GalSimInterface import GalSimStars, DoubleGaussianPSF
from lsst.obs.lsstSim import LsstSimMapper #this is to model the LSST camera

class testGalSimStars(GalSimStars):
    bandpassNames = ['u', 'r']
    camera = LsstSimMapper().camera
    PSF = DoubleGaussianPSF()

In [ ]:
#use our ObservationMetaDataGenerator from above
obsMetaDataResults = generator.getObservationMetaData(limit=1, fieldRA = (-5.0, 5.0),
                                                      boundType = 'circle', boundLength = 0.01)

myTestCat = testGalSimStars(starTableConnection, obs_metadata=obsMetaDataResults[0])
myTestCat.write_catalog('galsim_test_catalog.txt')
myTestCat.write_images(nameRoot = 'testImage')

This code generates a series of FITS images, one for each detector in our camera. The files are named testImage_detectorName_filterName.fits.  It also produced a catalog galsim_test_catalog.txt that contains information about the camera in the header as well as information about each object, namely:

* Its source type ('point' or 'sersic')

* A unique identifier

* The name of the chip its centroid fell on

* Its location on the pupil

* The name of the file containing its SED

* Its major and minor axes

* The index of its Sersic Profile

* Its half light radius

* Its position angle

* A list of all of the detectors it may have illuminated

<b> Customizing the GalSim Catalog classes </b>

As with all of the `InstanceCatalog` daughter classes we have seen so far, the GalSim Catalog is customized using class member variables.  Specifically, you may want to specify


* `bandpassNames` = a list of the names of the bandpasses you are observing through

* `bandpassDir` = a string denoting the directory in which your bandpass throughput files live

* `bandpassRoot` = a string that is the root of the name of your bandpass files

* `componentList` = a list of filenames (files kept in `bandpassDir`) specifying the throughputs of the individual (non-filter) components of your camera (i.e. mirrors, lenses, etc.)

* `atmoTransmissionName` = the name of a file (also kept in `bandpassDir`) specifying the transmissivity of the atmosphere

* `skySEDname` = the name of a file (also kept in `bandpassDir`) representing the emission spectrum of the dark sky in ergs/cm^2/s/nm (note: this is not yet used by the code, but is required as a placeholder).

All of the above default to files consistent with the LSST design.

The `GalSimCatalog` assumes that your bandpass throughput files are stored in files with the naming convention:

    for bpn in bandpass_names:
        fileName = self.bandpassDir + '/' + self.bandpassRoot + '_' + bpn + '.dat'

It reads in these files and convolves them with the hardware throughputs from `componentList` and the atmospherica transmissivity from `atmoTransmissionName` to produce the total throughput of your instrument.

Other user-specified variables are:

* `camera` = a member variable containing an instaniation of an afwCamerGeom camera object.  This defaults to a small, test camera.  The code above shows how to specify the LSST camera.


* `PSF` = a member variable containing a class describing the point spread function for your catalog.  This must be a class that inherits from PSFbase and defines certain methods as described in the PSFbase docstring.


In [ ]:
from lsst.sims.GalSimInterface import PSFbase

help(PSFbase)

Noise will be handled similarly, by assigning noise class instantiation to the member variable `noise_and_background`.  An example of this is provided in the class `ExampleCCDNoise`.

In [ ]:
from lsst.sims.GalSimInterface import ExampleCCDNoise

help(ExampleCCDNoise)

As in the case of PhoSim, we are presented with the challenge of how to generate images containing different types of astronomical object when the `CatalogDBObject` classes are segregated the way they are.  In this case, the `GalSimCatalog` classes containg a method `copyGalSimInterpreter` which allows you to pass the `GalSimInterpreter` and `camera` instantiation from one `GalSimCatalog` to another.  Since the `GalSimInterpreter` is the code that actually handles writing FITS images, this allows you to add multiple catalogs of astronomical objects to the same set of FITS images.  This is demonstrated in the script

    sims_GalSimInterface/examples/galSimCompoundGenerator.py